In [38]:
import numpy as np
import networkx as nx
import h5py

from decoder_performance import compute_logical_error_rate
from logical_operators import get_logical_operators_by_pivoting
from ldpc.bposd_decoder import BpOsdDecoder
import ldpc.code_util
import ldpc
from scipy.sparse import csr_matrix
import networkx as nx
import numpy as np
from optimization.experiments_settings import tanner_graph_to_parity_check_matrix
from basic_css_code import construct_HGP_code
import sys
import os

def count_girth_4(state: nx.MultiGraph):
    """Counts the number of cycles of length 4 in the Tanner graph."""
    # In a bipartite graph, a cycle of length 4 is a 'quadrilateral'
    # This can be found by looking at nodes with common neighbors
    cycles = 0
    # We only need to iterate over one side of the bipartite graph (e.g., check nodes)
    # Check nodes in your project are usually identified by their type or index
    nodes = [n for n, d in state.nodes(data=True) if d.get('type') == 'check']
    
    # If types aren't explicitly set, you can use the node indices 
    # corresponding to the rows of your parity check matrix H.
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i+1:]:
            # Common neighbors between two check nodes
            shared = set(state.neighbors(node1)) & set(state.neighbors(node2))
            if len(shared) >= 2:
                # Number of 4-cycles formed by these two nodes is 'n choose 2'
                n = len(shared)
                cycles += n * (n - 1) // 2
    return cycles

def count_girth_6(state: nx.Graph):
    """
    Counts the number of cycles of length 6 in a bipartite Tanner graph.
    Formula: A 6-cycle is formed by 3 distinct checks and 3 distinct variables.
    """
    count = 0
    # Isolate check nodes
    check_nodes = [n for n, d in state.nodes(data=True) if d.get('type') == 'check']
    
    for c1 in check_nodes:
        # Get neighbors (variables) of c1
        vars_of_c1 = list(state.neighbors(c1))
        
        # Get other checks connected to those variables
        for i, v1 in enumerate(vars_of_c1):
            for v2 in vars_of_c1[i+1:]:
                # Find checks (other than c1) that connect v1 and v2
                common_checks = set(state.neighbors(v1)) & set(state.neighbors(v2))
                common_checks.discard(c1)
                
                for c2 in common_checks:
                    # To complete a 6-cycle, we need a third variable v3 
                    # that connects c2 to c1 through a path of length 2
                    # But actually, counting common neighbors of checks is easier:
                    pass

    # A more reliable 'Matrix' style approach for small/medium graphs:
    adj = nx.to_numpy_array(state)
    # The trace of the adjacency matrix raised to the 6th power 
    # relates to the number of 6-cycles.
    # num_6_cycles = trace(A^6 - terms for 4-cycles) / 12
    # Since your 4-cycle count is 0, this simplifies!
    
    A6 = np.linalg.matrix_power(adj, 6)
    # Each 6-cycle is counted 12 times (6 nodes * 2 directions)
    return int(np.trace(A6) / 12)

import numpy as np
from itertools import combinations

def count_all_min_weight_logical_ops(Lz, distance):
    """
    Exhaustively searches all 2^k linear combinations of the logical basis 
    to find the total number of operators at a specific weight.
    """
    k, n = Lz.shape
    total_min_weight_ops = 0
    
    # We iterate through combinations of rows (1 to k)
    # Note: This is O(2^k), so only use for small k!
    for r in range(1, k + 1):
        for combo in combinations(range(k), r):
            # XOR the selected rows together
            op = np.bitwise_xor.reduce(Lz[list(combo)], axis=0)
            if np.sum(op) == distance:
                total_min_weight_ops += 1
                
    return total_min_weight_ops

def get_Lz_and_d_from_state(state: nx.MultiGraph):
    """
    Extracts the logical Z operator basis matrix Lz from the given Tanner graph state.
    """
    H = tanner_graph_to_parity_check_matrix(state)

    csr_H = csr_matrix(H, dtype=np.uint8)
    base_payload = compute_hgp_code_parameters(H=csr_H)
    print(base_payload)
    d_classical = base_payload['d_classical']
    d_T_classical = base_payload['d_T_classical']

    min_classical_distance = min(d_classical, d_T_classical)
        
    Hx, Hz = construct_HGP_code(H)

    Lx, Lz = get_logical_operators_by_pivoting(Hx, Hz)

    print(f"Lz shape: {Lz.shape}")

    return Lz, d_classical

def compute_hgp_code_parameters(H: csr_matrix):
    csr_H = H
    r_classical = ldpc.mod2.rank(csr_H)
    n_classical, k_classical, _ = ldpc.code_util.compute_code_parameters(csr_H)
    d_classical = ldpc.code_util.compute_exact_code_distance(csr_H)
    print(f"H: [{n_classical}, {k_classical}, {d_classical}]")
    if k_classical == n_classical - csr_H.shape[0]:
        n_T_classical = csr_H.shape[0]
        k_T_classical = n_T_classical - r_classical  # k will be 0 if full rank
        d_T_classical = np.inf
    else:
        n_T_classical, k_T_classical, d_T_classical = ldpc.code_util.compute_code_parameters(
            csr_matrix(H.T, dtype=np.uint8))
    print(f"H^T: [{n_T_classical}, {k_T_classical}, {d_T_classical}]")

    Hx, Hz = construct_HGP_code(H)

    n_quantum = n_classical**2 + n_T_classical**2
    k_quantum = k_classical**2 + k_T_classical**2
    d_quantum = min(d_classical, d_T_classical)

    print(f"[[{n_quantum}, {k_quantum}, {d_quantum}]]")

    return {
        "n_classical": n_classical,
        "k_classical": k_classical,
        "d_classical": d_classical,
        "n_T_classical": n_T_classical,
        "k_T_classical": k_T_classical,
        "d_T_classical": d_T_classical,
        "rank_H": r_classical,
        "n_quantum": n_quantum,
        "k_quantum": k_quantum,
        "d_quantum": d_quantum,
    }

def get_logical_multiplicity(Lz, distance):
    """Counts how many logical basis operators are at the minimum distance."""
    if Lz.shape[0] == 0:
        return 0
    row_weights = np.sum(Lz, axis=1)
    # Count operators exactly equal to the distance
    multiplicity = np.sum(row_weights == distance)
    return int(multiplicity)

In [41]:
def count_combinations_weight(Lz, distance, max_combo=2):
    k = Lz.shape[0]
    count = 0
    from itertools import combinations
    for r in range(2, max_combo + 1):
        for combo in combinations(range(k), r):
            res = np.bitwise_xor.reduce(Lz[list(combo)], axis=0)
            if np.sum(res) == distance:
                count += 1
    return count

In [39]:
import h5py
from optimization.analyze_codes.decoder_performance_from_state import evaluate_performance_of_state
from optimization.experiments_settings import from_edgelist

filepath = "optimization/results/beam_search_d_first_L3.hdf5"
code_name = "[1600,64]"  # Update this key if needed
p = 0.03
MC_budget = int(5 * 1e6)

with h5py.File(filepath, "r") as f:
    grp = f[code_name]
    # Load datasets
    best_state_edge_list = grp["best_state"][:]

    best_state = from_edgelist(best_state_edge_list)

    min_cost = grp.attrs.get('min_cost', None)

    print(f"cost saved: {min_cost}")

    print(best_state_edge_list)

    cycles = count_girth_6(best_state)

    Lz, d_classical = get_Lz_and_d_from_state(best_state)

    print(f"Logical Z operators: {Lz}")
    print(f"Classical distance: {d_classical}")

    for i in range(10):
        logical_multiplicity = get_logical_multiplicity(Lz, d_classical + i)

        print(f"Number of logical operators at weight {d_classical + i}: {logical_multiplicity}")

    print(f"Number of 6-cycles in the best state: {cycles}")
    # cost_result = evaluate_performance_of_state(
    #     state=best_state, p_vals=[p], MC_budget=MC_budget, canskip=False)

cost saved: 0.0008
[[ 0 24  0 32  0 39  0 49  1 24  1 33  1 40  1 48  1 55  2 24  2 34  2 41
   2 50  3 32  3 42  3 49  3 50  3 53  4 25  4 35  4 41  4 51  5 25  5 36
   5 43  5 48  6 26  6 32  6 44  6 51  7 26  7 36  7 36  7 37  8 26  8 38
   8 41  8 53  9 27  9 33  9 42  9 52 10 27 10 35 10 37 10 44 11 27 11 43
  11 53 11 55 12 28 12 33 12 45 12 51 13 28 13 40 13 44 13 50 14 28 14 38
  14 46 14 49 15 29 15 34 15 42 15 46 16 29 16 47 16 52 16 52 17 25 17 29
  17 37 17 45 18 30 18 34 18 43 18 54 19 30 19 38 19 47 19 55 20 30 20 39
  20 45 21 31 21 35 21 46 21 54 22 31 22 39 22 48 22 54 23 31 23 40 23 47]]
H: [32, 8, 11]
H^T: [24, 0, inf]
[[1600, 64, 11]]
{'n_classical': 32, 'k_classical': 8, 'd_classical': 11, 'n_T_classical': 24, 'k_T_classical': 0, 'd_T_classical': inf, 'rank_H': 24, 'n_quantum': 1600, 'k_quantum': 64, 'd_quantum': 11}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/ldpc/code_util/code_util.py:164: UserWarning: This function has exponential complexity. Not recommended for large pcms. Use the                            'ldpc.code_util.estimate_code_distance' function instead.
  warnings.warn(


Hx, Hz, Lx, Lz: (768, 1600), (768, 1600), (64, 1600), (64, 1600)
Lz shape: (64, 1600)
Logical Z operators: [[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Classical distance: 11
Number of logical operators at weight 11: 8
Number of logical operators at weight 12: 16
Number of logical operators at weight 13: 16
Number of logical operators at weight 14: 8
Number of logical operators at weight 15: 8
Number of logical operators at weight 16: 0
Number of logical operators at weight 17: 8
Number of logical operators at weight 18: 0
Number of logical operators at weight 19: 0
Number of logical operators at weight 20: 0
Number of 6-cycles in the best state: 809


In [42]:
from optimization.experiments_settings import codes, path_to_initial_codes, textfiles
from optimization.experiments_settings import generate_neighbor_highlight, from_edgelist, load_tanner_graph, parse_edgelist

C = 2

initial_state = load_tanner_graph(path_to_initial_codes + textfiles[C])

edge_list = parse_edgelist(initial_state)
print(edge_list)

Lz, d_classical = get_Lz_and_d_from_state(initial_state)

print(f"Logical Z operators: {Lz}")
print(f"Classical distance: {d_classical}")

for i in range(15):
    logical_multiplicity = get_logical_multiplicity(Lz, d_classical + i)

    print(f"Number of logical operators at weight {d_classical + i}: {logical_multiplicity}")

count = count_combinations_weight(Lz, d_classical, max_combo=3)

print(f"Number of combinations of up to 3 logical basis operators that yield weight {d_classical}: {count}")

cycles = count_girth_6(initial_state)

print(f"Number of 6-cycles in the initial state: {cycles}")

[ 0 24  0 32  0 39  0 49  1 24  1 33  1 40  1 48  1 55  2 24  2 34  2 41
  2 50  3 25  3 32  3 42  3 50  3 55  4 25  4 35  4 41  4 51  5 25  5 36
  5 43  5 48  6 26  6 32  6 44  6 51  7 26  7 37  7 40  7 52  8 26  8 38
  8 41  8 53  9 27  9 33  9 42  9 52 10 27 10 35 10 44 10 53 11 27 11 37
 11 43 11 49 12 28 12 33 12 45 12 51 13 28 13 36 13 44 13 50 14 28 14 38
 14 46 14 49 15 29 15 34 15 42 15 46 16 29 16 36 16 47 16 52 17 29 17 37
 17 45 17 54 18 30 18 34 18 43 18 54 19 30 19 38 19 47 19 55 20 30 20 39
 20 45 21 31 21 35 21 46 21 54 22 31 22 39 22 48 22 53 23 31 23 40 23 47]
H: [32, 8, 6]
H^T: [24, 0, inf]
[[1600, 64, 6]]
{'n_classical': 32, 'k_classical': 8, 'd_classical': 6, 'n_T_classical': 24, 'k_T_classical': 0, 'd_T_classical': inf, 'rank_H': 24, 'n_quantum': 1600, 'k_quantum': 64, 'd_quantum': 6}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/ldpc/code_util/code_util.py:164: UserWarning: This function has exponential complexity. Not recommended for large pcms. Use the                            'ldpc.code_util.estimate_code_distance' function instead.
  warnings.warn(


Hx, Hz, Lx, Lz: (768, 1600), (768, 1600), (64, 1600), (64, 1600)
Lz shape: (64, 1600)
Logical Z operators: [[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Classical distance: 6
Number of logical operators at weight 6: 0
Number of logical operators at weight 7: 0
Number of logical operators at weight 8: 0
Number of logical operators at weight 9: 0
Number of logical operators at weight 10: 0
Number of logical operators at weight 11: 0
Number of logical operators at weight 12: 0
Number of logical operators at weight 13: 0
Number of logical operators at weight 14: 16
Number of logical operators at weight 15: 0
Number of logical operators at weight 16: 48
Number of logical operators at weight 17: 0
Number of logical operators at weight 18: 0
Number of logical operators at weight 19: 0
Number of logical operators at weight 20: 0
Number of combinations of up to 3 logical basis operators that yield weight 6: 8
Number of 6-

In [43]:
from optimization.experiments_settings import codes, path_to_initial_codes, textfiles
from optimization.experiments_settings import generate_neighbor_highlight, from_edgelist, load_tanner_graph, parse_edgelist

C = 0

initial_state = load_tanner_graph(path_to_initial_codes + textfiles[C])

edge_list = parse_edgelist(initial_state)
print(edge_list)

Lz, d_classical = get_Lz_and_d_from_state(initial_state)

print(f"Logical Z operators: {Lz}")
print(f"Classical distance: {d_classical}")

for i in range(15):
    logical_multiplicity = get_logical_multiplicity(Lz, d_classical + i)

    print(f"Number of logical operators at weight {d_classical + i}: {logical_multiplicity}")

count = count_combinations_weight(Lz, d_classical, max_combo=3)

print(f"Number of combinations of up to 3 logical basis operators that yield weight {d_classical}: {count}")

cycles = count_girth_6(initial_state)

print(f"Number of 6-cycles in the initial state: {cycles}")

[ 0 15  0 20  0 24  0 30  1 15  1 21  1 25  1 31  2 15  2 22  2 26  2 32
  2 34  3 16  3 20  3 26  3 31  4 16  4 22  4 25  4 30  5 16  5 23  5 27
  5 32  6 17  6 20  6 27  6 33  7 17  7 22  7 28  7 31  8 17  8 23  8 29
  8 30  9 18  9 21  9 28  9 32 10 18 10 23 10 26 10 33 11 18 11 24 11 29
 11 34 12 19 12 21 12 27 12 34 13 19 13 24 13 28 13 33 14 19 14 25 14 29]
H: [20, 5, 6]
H^T: [15, 0, inf]
[[625, 25, 6]]
{'n_classical': 20, 'k_classical': 5, 'd_classical': 6, 'n_T_classical': 15, 'k_T_classical': 0, 'd_T_classical': inf, 'rank_H': 15, 'n_quantum': 625, 'k_quantum': 25, 'd_quantum': 6}
Hx, Hz, Lx, Lz: (300, 625), (300, 625), (25, 625), (25, 625)
Lz shape: (25, 625)
Logical Z operators: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Classical distance: 6
Number of logical operators at weight 6: 5
Number of logical operators at weight 7: 0
Number of logical operators at weight 8: 10
Number of logical operators a

In [44]:
from optimization.experiments_settings import codes, path_to_initial_codes, textfiles
from optimization.experiments_settings import generate_neighbor_highlight, from_edgelist, load_tanner_graph, parse_edgelist

C = 1

initial_state = load_tanner_graph(path_to_initial_codes + textfiles[C])

edge_list = parse_edgelist(initial_state)
print(edge_list)

Lz, d_classical = get_Lz_and_d_from_state(initial_state)

print(f"Logical Z operators: {Lz}")
print(f"Classical distance: {d_classical}")

for i in range(15):
    logical_multiplicity = get_logical_multiplicity(Lz, d_classical + i)

    print(f"Number of logical operators at weight {d_classical + i}: {logical_multiplicity}")

count = count_combinations_weight(Lz, d_classical, max_combo=3)

print(f"Number of combinations of up to 3 logical basis operators that yield weight {d_classical}: {count}")

cycles = count_girth_6(initial_state)

print(f"Number of 6-cycles in the initial state: {cycles}")

[ 0 21  0 28  0 35  0 42  0 48  1 21  1 29  1 36  1 41  2 21  2 30  2 37
  2 43  3 22  3 28  3 38  3 43  3 47  4 22  4 31  4 36  4 42  5 22  5 32
  5 37  5 44  6 23  6 28  6 39  6 44  7 23  7 33  7 36  7 43  8 23  8 34
  8 37  8 45  9 24  9 29  9 38  9 45 10 24 10 31 10 35 10 46 11 24 11 33
 11 40 11 44 12 25 12 29 12 39 12 46 13 25 13 32 13 35 13 45 14 25 14 34
 14 40 14 47 15 26 15 30 15 38 15 46 16 26 16 31 16 39 16 47 17 26 17 34
 17 41 17 48 18 27 18 30 18 40 18 48 19 27 19 32 19 41 20 27 20 33 20 42]
H: [28, 8, 6]
H^T: [21, 1, 12]
[[1225, 65, 6]]
{'n_classical': 28, 'k_classical': 8, 'd_classical': 6, 'n_T_classical': 21, 'k_T_classical': 1, 'd_T_classical': 12, 'rank_H': 20, 'n_quantum': 1225, 'k_quantum': 65, 'd_quantum': 6}
Hx, Hz, Lx, Lz: (588, 1225), (588, 1225), (65, 1225), (65, 1225)
Lz shape: (65, 1225)
Logical Z operators: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Classical distance: 6
Number o

In [45]:
from optimization.experiments_settings import codes, path_to_initial_codes, textfiles
from optimization.experiments_settings import generate_neighbor_highlight, from_edgelist, load_tanner_graph, parse_edgelist

C = 3

initial_state = load_tanner_graph(path_to_initial_codes + textfiles[C])

edge_list = parse_edgelist(initial_state)
print(edge_list)

Lz, d_classical = get_Lz_and_d_from_state(initial_state)

print(f"Logical Z operators: {Lz}")
print(f"Classical distance: {d_classical}")

for i in range(15):
    logical_multiplicity = get_logical_multiplicity(Lz, d_classical + i)

    print(f"Number of logical operators at weight {d_classical + i}: {logical_multiplicity}")

count = count_combinations_weight(Lz, d_classical, max_combo=3)

print(f"Number of combinations of up to 3 logical basis operators that yield weight {d_classical}: {count}")

cycles = count_girth_6(initial_state)

print(f"Number of 6-cycles in the initial state: {cycles}")

[ 0 27  0 36  0 44  0 55  1 27  1 37  1 46  1 56  2 27  2 38  2 45  2 53
  3 28  3 36  3 47  3 56  4 28  4 39  4 46  4 55  5 28  5 40  5 48  5 54
  6 29  6 36  6 49  6 57  7 29  7 41  7 46  7 58  8 29  8 42  8 48  8 59
  9 30  9 37  9 49  9 52 10 30 10 40 10 50 10 57 11 30 11 41 11 45 11 59
 12 31 12 37 12 51 12 59 13 31 13 42 13 50 13 60 14 31 14 43 14 47 14 58
 15 32 15 38 15 47 15 60 16 32 16 40 16 51 16 55 17 32 17 42 17 52 17 61
 18 33 18 38 18 48 18 61 19 33 19 41 19 51 19 62 20 33 20 43 20 49 21 34
 21 39 21 53 21 57 22 34 22 43 22 54 22 56 22 61 23 34 23 44 23 50 23 62
 24 35 24 39 24 52 24 62 25 35 25 44 25 53 25 58 26 35 26 45 26 54 26 60]
H: [36, 9, 10]
H^T: [27, 0, inf]
[[2025, 81, 10]]
{'n_classical': 36, 'k_classical': 9, 'd_classical': 10, 'n_T_classical': 27, 'k_T_classical': 0, 'd_T_classical': inf, 'rank_H': 27, 'n_quantum': 2025, 'k_quantum': 81, 'd_quantum': 10}
Hx, Hz, Lx, Lz: (972, 2025), (972, 2025), (81, 2025), (81, 2025)
Lz shape: (81, 2025)
Logical Z operators